In [8]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [11]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re

# 파일 데이터 가져와서 라벨 붙이기 

In [14]:
# 경로확인 

In [13]:
pwd

'C:\\Users\\bitcamp\\Project4'

In [ ]:
# 데이터가 있는 경로로 이동

In [17]:
cd C:\Users\bitcamp\Project4\data\CNBC뉴스분류-20220425T095102Z-001\CNBC뉴스분류

C:\Users\bitcamp\Project4\data\CNBC뉴스분류-20220425T095102Z-001\CNBC뉴스분류


In [19]:
# 데이터 리스팅
import os
# csv파일 경로 리스팅 
data_lis = os.listdir('./')
data_lis

['labeled_news_oil_0_100_oil_price.csv.csv',
 'labeled_news_oil_100_200_oil_price.csv.csv',
 'labeled_news_oil_200_300_oil_price.csv.csv',
 'labeled_news_oil_300_400_oil_price.csv.csv',
 'labeled_news_oil_400_500_oil_price.csv.csv',
 'labeled_news_oil_500_600_oil_price.csv.csv',
 'labeled_news_oil_600_700_oil_price.csv.csv',
 'labeled_news_oil_700_end_oil_price.csv.csv',
 'labeled_news_semiconductor_0_100_semicondutor.csv.csv',
 'labeled_news_semiconductor_200_300_semicondutor.csv.csv',
 'labeled_news_semiconductor_300_400_semicondutor.csv.csv',
 'labeled_news_semiconductor_400_500_semicondutor.csv.csv',
 'labeled_news_semiconductor_500_600_semicondutor.csv.csv',
 'labeled_news_semiconductor_600_700_semicondutor.csv.csv',
 'labeled_news_semiconductor_700_end_semicondutor.csv.csv',
 'labeled_news_ship_0_100_ship_building.csv.csv',
 'labeled_news_ship_200_300_ship_building.csv.csv',
 'labeled_news_ship_300_400_ship_building.csv.csv',
 'labeled_news_ship_400_500_ship_building.csv.csv',
 '

In [35]:
# 반복문 이용해서 데이터 합치기 
# csv파일 여러개 읽어와서 합치기 
df_cnbc = pd.DataFrame()

for files in data_lis:
    # advertisement 빼고 합치기 
    df = pd.read_csv(files,index_col=0)
    df_cnbc = pd.concat([df_cnbc,df])    

In [36]:
df = df_cnbc

In [41]:
df = df.sort_values('date')

In [44]:
df

,date,content,label
23,2013-09-09T04:06:00+0000,"[' Bright colors, funky textures and personal...",102
15,2013-09-09T12:34:19+0000,[' You already can invest your retirement mon...,-1
60,2013-09-09T20:50:18+0000,[' The European automotive market has come un...,104
5,2013-09-10T16:08:23+0000,"[' If at first you don’t succeed, order a sec...",104
50,2013-09-17T19:10:42+0000,[' Chinese reverse merger companies listed on...,-1
...,...,...,...
51,2022-04-22T12:34:04+0000,['American Eagle Outfitters wants to be more l...,100
29,2022-04-22T19:01:26+0000,['The path to a mega fleet of autonomous vehic...,104
62,2022-04-22T20:20:03+0000,['WASHINGTON — From heavy artillery to tactica...,-1
19,2022-04-22T21:15:13+0000,['A new wave of single-stock exchange-traded f...,-1


In [ ]:
# 모델이랑 피클 저장할 경로로 바꿔주기
# ./카테고리분류모델피클 

In [43]:
cd C:\Users\bitcamp\Project4\카테고리분류모델피클

C:\Users\bitcamp\Project4\카테고리분류모델피클


# 토크나이징 하기 

In [45]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re

In [46]:
def preprocessing(docs):
    tokenizer = RegexpTokenizer('[\w]+')
    stop_words = stopwords.words('english')
    p_stemmer = PorterStemmer()  # 어근 복원 : runs, running, ran => run,  cars => car
#     p = re.compile("[0-9]+")
#     숫자제거해야됨
#     docs = [p.sub(i) for i in docs]
    docs = BeautifulSoup(docs, "html5lib").get_text()
    low = docs.lower()
    tokens = tokenizer.tokenize(low)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    stopped_tokens = [re.sub('[^a-zA-Z0-9]',' ',i) for i in stopped_tokens]
    stemmer_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    one_rmv = [w for w in stemmer_tokens if len(w)>1]
  
    return one_rmv

In [50]:
# 함수 적용 
df['tokened'] = df['content'].apply(preprocessing)

# @@토큰화 하기전 컬럼 보존 @@

In [51]:
df

,date,content,label,tokened
23,2013-09-09T04:06:00+0000,"[' Bright colors, funky textures and personal...",102,"[bright, color, funki, textur, person, come, s..."
15,2013-09-09T12:34:19+0000,[' You already can invest your retirement mon...,-1,"[alreadi, invest, retir, money, kid, colleg, s..."
60,2013-09-09T20:50:18+0000,[' The European automotive market has come un...,104,"[european, automot, market, come, unplug, rece..."
5,2013-09-10T16:08:23+0000,"[' If at first you don’t succeed, order a sec...",104,"[first, succeed, order, second, recal, rav4, g..."
50,2013-09-17T19:10:42+0000,[' Chinese reverse merger companies listed on...,-1,"[chines, revers, merger, compani, list, exchan..."
...,...,...,...,...
51,2022-04-22T12:34:04+0000,['American Eagle Outfitters wants to be more l...,100,"[american, eagl, outfitt, want, like, amazon, ..."
29,2022-04-22T19:01:26+0000,['The path to a mega fleet of autonomous vehic...,104,"[path, mega, fleet, autonom, vehicl, tesla, co..."
62,2022-04-22T20:20:03+0000,['WASHINGTON — From heavy artillery to tactica...,-1,"[washington, heavi, artilleri, tactic, drone, ..."
19,2022-04-22T21:15:13+0000,['A new wave of single-stock exchange-traded f...,-1,"[new, wave, singl, stock, exchang, trade, fund..."


# 전처리 하기

In [52]:
# 전처리 할 변수 만들어 주기 
preprocessed_sentences = df['tokened']

In [53]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
# tokenizer.fit_on_texts(preprocessed_sentences) 
vocab_size = 10000
tokenizer = Tokenizer(num_words = vocab_size + 1) 
tokenizer.fit_on_texts(preprocessed_sentences)

# 상위 1만개 단어 사용 해서 빈도수 기준으로 단어 집합 생성 하기 

In [54]:
#fit_on_texts로 토큰화 한 것을 pickle로 저장 
import pickle
import re

In [55]:
# pickle파일이 현재 디렉토리에 저장됨 
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)

In [56]:
# 단어를 시퀀스 형태로 변환해주기 
x = tokenizer.texts_to_sequences(preprocessed_sentences)

In [57]:
# 훈련시킬 할 라벨 지정 
y = df['label']

# 모델 돌리기 

In [58]:
# 훈련시킬 데이터 셋 나누기
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=111)

In [60]:
# 기사최대길이로 패딩해주기 
max_len = max([len(l) for l in df['tokened']])

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test,maxlen=max_len)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [65]:
# LSTM 모델 구성

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 128
hidden_units = 128
num_classes = 21

with tf.device('/GPU:0'):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(hidden_units))
    model.add(Dense(num_classes, activation='softmax'))

In [67]:
df['label'].value_counts()

-1      1134
 100     554
 104     479
 1       250
 3       168
 102     162
 12       31
 13       28
 15       27
 17       25
 21       21
 19       21
 20       19
 22       18
 28       14
 24       14
 101      13
 103      12
 32       11
 31        9
 29        8
Name: label, dtype: int64

In [66]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
# 에포크20때까지 val_acc가 개선되지않으면 스톱 
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# 제일 훈련이 잘된것을 현재경로에 저장 


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(x_train, y_train, batch_size=32, epochs=30, callbacks=[es, mc], validation_data=(x_test, y_test) )

Epoch 1/30


ValueError: in user code:

    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\bitcamp\anaconda3\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "C:\Users\bitcamp\anaconda3\

In [ ]:
# 모델 시각화 
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()